# Bayesian statistics


- work in progress
- data cleaning demo
- Bayesian analysis


In [1]:
# Libraries
import math
import numpy as np
import pandas as pd
import re
from skimpy import clean_columns

In [2]:
# Helpers
def excol_to_index(excel_col):
    '''Input an Excel column code and return the index'''
    if (excel_col is None) or (excel_col == ""):
        print("Input must be a non-empty character string.")
        return None

    excel_col = str(excel_col).upper().strip()
    
    if not excel_col.isalpha():
        print("Input must contain only letters.")
        return None
    
    index = 0
    for c in excel_col:
        index = index * 26 + (ord(c) - ord('A') + 1)

    return index

In [3]:
# Raw data
root_folder = r"D:\data\electoral_commission_public_attitudes"

ec_ds_raw = pd.read_excel(
    root_folder + r"\Electoral Commission Winter Tracker 2018-2025 Historical Data v3.xlsx", 
    sheet_name="Counts", 
    header=None,
    skiprows=7,
    nrows=None
)
ec_ds_raw = clean_columns(ec_ds_raw)

## Survey data

### Reference data frame

In [26]:
# Extract a df of indexed questions
ec_qs = pd.read_excel(
    root_folder + r"\Electoral Commission Winter Tracker 2018-2025 Historical Data v3.xlsx", 
    sheet_name="Counts", 
    header=None,
    skiprows=7,
    nrows=None
)
ec_qs = clean_columns(ec_qs)
ec_qs = ec_qs.iloc[:,0:2]

    # Add index
ec_qs["start_index"] = ec_qs.index
ec_qs.rename({"0": "question"}, axis=1, inplace=True)

    # Filter questions
ec_qs = ec_qs[ec_qs["question"].notna() & ec_qs.iloc[:,1].isna()]

    # Add lagged index
ec_qs["end_index"] = ec_qs["start_index"].shift(-1).fillna(-1).astype("int32") - 1

    # Tidy up
ec_qs = ec_qs[["start_index", "end_index", "question"]]
ec_qs = ec_qs.reset_index(drop=True)

ec_qs.head(n=5)

,start_index,end_index,question
0,0,19,year. Year
1,20,20,How knowledgeable would you say you are about ...
2,21,34,S7b_1. UK Parliament (General Elections)
3,35,48,S7b_2. Local elections
4,49,62,S7b_3. Senedd/Northern Ireland assembly/Scotti...


## Dataset

In [5]:
# Construct a header
ec_header_raw = pd.read_excel(
    root_folder + r"\Electoral Commission Winter Tracker 2018-2025 Historical Data v3.xlsx", 
    sheet_name="Counts", 
    header=None,
    skiprows=4,
    nrows=3
)
ec_header_raw = clean_columns(ec_header_raw)

row1 = ec_header_raw.iloc[0,].dropna().astype("string") # Top header
row2 = ec_header_raw.iloc[1,].dropna().astype("int32") # Years
row3 = ec_header_raw.iloc[2,].dropna().astype("string") # Excel-style index

row1 = row1.str.replace("[ /+]", "", regex=True)
row1 = row1.str.replace("[-:]", "_", regex=True)
row1 = tuple(map(lambda x: x.lower(), row1))

row1

('total',
 'year',
 'gender_male',
 'gender_female',
 'age_16_24',
 'age_25_34',
 'age_35_44',
 'age_45_64',
 'age_65_74',
 'age_75',
 'socialgrade_ab',
 'socialgrade_c1',
 'socialgrade_c2',
 'socialgrade_de',
 'socialgrade_net_abc1',
 'socialgrade_net_c2de',
 'nation_england',
 'nation_wales',
 'nation_scotland',
 'nation_northernireland',
 'nation_greatbritain')

In [6]:
    # Construct a new header
ec_cols = [f"{c}_{y}" for c in row1[1:] for y in range(min(row2), max(row2) + 1)]

ec_header = ["qa", "total", *ec_cols]
ec_header_dict = dict(zip(ec_ds_raw.columns, ec_header))

# Dimension check
len(ec_header) == ec_ds_raw.shape[1]

True

In [7]:
# Main dataset data frame
ec_ds = ec_ds_raw
ec_ds.rename(ec_header_dict, axis=1, inplace=True)

ec_ds[ec_header[1:]] = ec_ds[ec_header[1:]].apply(pd.to_numeric, errors="coerce")

ec_ds.head(n=10)

,qa,total,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023,year_2024,year_2025,...,nation_northernireland_2024,nation_northernireland_2025,nation_greatbritain_2018,nation_greatbritain_2019,nation_greatbritain_2020,nation_greatbritain_2021,nation_greatbritain_2022,nation_greatbritain_2023,nation_greatbritain_2024,nation_greatbritain_2025
0,year. Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unweighted base,31304.00,1300.00,1731.00,3201.0,3418.00,5486.00,4375.00,5874.00,5919.00,...,1000.00,1036.00,1100.00,1530.00,2701.00,2906.00,4928.00,3865.00,4874.00,4883.00
2,Base,31076.98,1300.00,1730.98,3201.0,3417.00,5260.00,4375.00,5874.00,5919.00,...,166.85,169.77,1264.91,1687.23,3111.37,3311.39,5102.20,4249.70,5707.15,5749.23
3,Effective base,17219.04,1136.44,1297.81,2283.0,2001.55,2376.28,2735.17,3039.85,2924.72,...,805.00,787.61,1080.48,1238.27,2164.91,1888.27,2245.74,2594.98,2878.38,2767.81
4,2018,1300.00,1300.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1264.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019,1730.98,NaN,1730.98,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1687.23,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020,3201.00,NaN,NaN,3201.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3111.37,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Dimensions of the main dataset df
print("{r} rows and {c} cols".format(r=ec_ds.shape[0], c=ec_ds.shape[1]))

6009 rows and 162 cols


## Question-specific subsets

In [39]:
# Q9D2. How did you register to vote (2022)?
    # Get the question row-range
q9d2 = "How did you register to vote"
q9d2_mask = ec_qs["question"].str.contains(q9d2, na=False)
q9d2_start = ec_qs.loc[q9d2_mask, "start_index"].astype(int).tolist()[0]
q9d2_end = ec_qs.loc[q9d2_mask, "end_index"].astype(int).tolist()[0]

print("Index range:")
(q9d2_start, q9d2_end)

Index range:


(1472, 1481)

In [62]:
q9d2_df = ec_ds.iloc[range(q9d2_start, q9d2_end), ]
q9d2_df = q9d2_df[["qa", "total"] + [c for c in q9d2_df.columns.tolist() if "year" in c]]
q9d2_df = q9d2_df[["qa", "year_2022"]]

q9d2_df["qa"] = (q9d2_df["qa"]
                 .str.strip()
                 .str.lower()
                 .str.replace(" ", "_", regex=False)
                 .str.replace("'", "", regex=False))

q9d2_df.loc[q9d2_df["qa"].str.contains("using", na=False), "qa"] = "paper"
q9d2_df.loc[q9d2_df["qa"].str.contains("cant", na=False), "qa"] = "neither"

q9d2_df = q9d2_df[q9d2_df["qa"].isin(["online", "paper", "neither"])]
q9d2_df["year_2022"] = q9d2_df["year_2022"].astype("int32")

q9d2_df

,qa,year_2022
1476,online,50
1478,paper,7
1480,neither,2


In [60]:
# Q23D. A deepfake video is a media in which a person in an existing video is replaced with someone else's likeness.Have you seen a deepfake video in the last year?
    # Get the question row-range
q23d = "A deepfake video is a media in which a person"
q23d_idx = ec_qs[ec_qs["question"].str.contains(q23d, na=False)].index
q23d_start = ec_qs["start_index"][q23d_idx].iat[0]
q23d_end = ec_qs["end_index"][q23d_idx].iat[0]

print("Index range:")
(q23d_start, q23d_end)

Index range:


(np.int64(3510), np.int32(3519))

In [79]:
q23d_df = ec_ds.iloc[range(q23d_start, q23d_end), ]
q23d_df = q23d_df[["qa", "total"] + [c for c in ec_ds.columns.tolist() if "year" in c]]
q23d_df = q23d_df[["qa", "year_2022", "year_2023"]]

q23d_df["qa"] = q23d_df["qa"].dropna().apply(
    lambda x: (
        temp := x.strip().lower(),
        temp := re.sub(" ", "_", temp),
        re.sub("'", "", temp)
    )[-1]
)

q23d_df["qa"] = np.select(
    [q23d_df["qa"].str.contains("dont", na=False)], 
    ["neither"], 
    default=q23d_df["qa"]
)

q23d_df = q23d_df[q23d_df["qa"].isin(["yes", "no", "neither"])]
q23d_cols = [c for c in q23d_df.columns.tolist() if "year" in c]
q23d_df[q23d_cols] = q23d_df[q23d_cols].astype("int32")

q23d_df

,qa,year_2022,year_2023
3514,yes,1054,856
3516,no,2466,2062
3518,neither,1739,1455
